In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve
import matplotlib
from matplotlib import animation
from clawpack.visclaw.JSAnimation import IPython_display

# Numerical methods

In [ ]:
def explicit_euler(f,t,u,h):
    """Take one explicit Euler step."""
    return u + h*f(t,u)

In [ ]:
def implicit_euler(f,t,u,h):
    """Take one implicit Euler step"""
    ff = lambda unew: unew - h*f(t,unew) - u
    unew = fsolve(ff,u)
    return unew

# The pendulum

Hamiltonian: $$H(p,q) = \frac{1}{2}p^2 - \cos(q)$$
Here $q$ is the angle the pendulum makes with the vertical while $p$ is the velocity.

The dynamics are governed by the equations

\begin{align}
    p'(t) & = - \sin(q) \\
    q'(t) & = p.
\end{align}

In [ ]:
def pendulum_rhs(t,pq):
    p = pq[0]
    q = pq[1]
    return np.array([-np.sin(q), p])

In [ ]:
def plot_trajectory(p,q,skip=10):

    plt.plot(p,q);
    axbnds = plt.axis('equal');
    plt.clf();

    # Set up plotting
    fig = plt.figure(figsize=(6,6)); axes = fig.add_subplot(111)
    line, = axes.plot([],[],lw=3)
    dot, = axes.plot([],[],'ok')
    axes.set_xlim((axbnds[0],axbnds[1])); axes.set_ylim((axbnds[2],axbnds[3]))

    def plot_frame(i):
        line.set_data(p[:i*skip],q[:i*skip])
        if i>0:
            dot.set_data(p[i*skip-1],q[i*skip-1])
        else:
            dot.set_data(p[0],q[0])
        axes.set_title('t='+str(i*h))
        fig.canvas.draw()
        return fig

    # Animate the solution
    return matplotlib.animation.FuncAnimation(fig, plot_frame,
                                       frames=len(p)/skip,
                                       interval=200,
                                       repeat=False)

In [ ]:
u0 = np.array([0,0.1])
uu = [u0]
h = 0.1
for k in range(400):
    uu.append(explicit_euler(pendulum_rhs,0,uu[-1],h))

In [ ]:
uu = np.array(uu)
p = uu[:,0]; q = uu[:,1]
plot_trajectory(p,q)

In [ ]:
def plot_energy(p,q):
    H = 0.5*p**2 - np.cos(q) + 1
    plt.plot(H);
    plt.title('Energy vs. time');
    
plot_energy(p,q)

In [ ]:
uu = [u0]
h = 0.1
for k in range(1000):
    uu.append(implicit_euler(pendulum_rhs,0,uu[-1],h))

In [ ]:
uu = np.array(uu)
p = uu[:,0]; q = uu[:,1]
plot_trajectory(p,q,skip=20)

In [ ]:
plot_energy(p,q)

# Symplectic Euler

\begin{align}
    p_{n+1} & = p_n + h f_1(p_{n+1},q_n) \\
    q_{n+1} & = q_n + h f_2(p_{n+1},q_n).
\end{align}

In [ ]:
def symplectic_euler_pendulum(t,u,h):
    """Take one symplectic Euler step for the pendulum problem."""
    p = u[0]
    q = u[1]
    pnew = p - h*np.sin(q)
    qnew = q + h*pnew
    return np.array([pnew,qnew])

In [ ]:
u0 = np.array([0,0.1])
uu = [u0]
h = 0.1
for k in range(1000):
    uu.append(symplectic_euler_pendulum(0,uu[-1],h))

In [ ]:
uu = np.array(uu)
p = uu[:,0]; q = uu[:,1]
plot_trajectory(p,q,skip=20)

In [ ]:
plot_energy(p,q)

# Stormer-Verlet (leapfrog or midpoint method)

\begin{align}
    p_{n+1/2} & = p_{n-1/2} + h f(q_n) \\
    q_{n+1} & = q_n + h p_{n+1/2}.
\end{align}

In [ ]:
def stormer_verlet_pendulum(t,u,h):
    """Take one Verlet step for the pendulum problem."""
    p = u[0]  # collocated at t_{n + 1/2}
    q = u[1]  # collocated at t_n
    qnew = q + h*p
    pnew = p - h*np.sin(qnew)
    return np.array([pnew,qnew])

In [ ]:
p0 = 0.
q0 = 0.1
h = 0.5
pnhalf = u0[0] - h/2. * np.sin(u0[1])
u0 = np.array([pnhalf,q0])
uu = [u0]

for k in range(1000):
    uu.append(stormer_verlet_pendulum(0,uu[-1],h))

In [ ]:
uu = np.array(uu)
p = uu[:,0];
p = np.insert(p,0,p0)
p = (p[1:]+p[:-1])/2.
q = uu[:,1]
plot_trajectory(p,q,skip=20)

In [ ]:
plot_energy(p,q)

The variation in the energy above is caused by the accuracy of our first (Euler) step.

In [ ]:
p0 = 0.
q0 = 0.1
h = 0.5
hsmall = h/10000.
v = np.array([p0,q0])
for k in range(10000):
     v = explicit_euler(pendulum_rhs,0,v,hsmall)
#uhalf = symplectic_euler_pendulum(0,np.array([p0,q0]),h/2.)
phalf = v[0]
u0 = np.array([phalf,q0])
uu = [u0]
for k in range(1000):
    uu.append(stormer_verlet_pendulum(0,uu[-1],h))

In [ ]:
uu = np.array(uu)
p = uu[:,0];
p = (p[1:]+p[:-1])/2.
p = np.insert(p,0,p0)
q = uu[:,1]
plot_trajectory(p,q,skip=20)

In [ ]:
plot_energy(p,q)

# Henon-Heiles

In [ ]:
def stormer_verlet_HH(t,u,h):
    """Take one Verlet step for the HH problem."""
    # u = [p1,p2,q1,q2]
    p1 = u[0]  # collocated at t_{n + 1/2}
    p2 = u[1]
    q1 = u[2]  # collocated at t_n
    q2 = u[3]
    q1new = q1 + h*p1
    q2new = q2 + h*p2
    p1new = p1 - h*(q1new-2*q1new*q2new)
    p2new = p2 - h*(q2new+q1new**2+q2new**2)
    return np.array([p1new,p2new,q1new,q2new])

In [ ]:
p10 = 0.
p20 = 0.
q10 = 0.0
q20 = 0.5
print(1./(0.5*(q10**2+q20**2+q10**2*q20 - 1./3.*q20**3)+0.5*(p10**2+p20**2)))
h = 0.1
p1half = p10 - h/2. * (q10-2*q10*q20)
p2half = p20 - h/2. * (q20+q10**2+q20**2)
u0 = np.array([p1half,p2half,q10,q20])
uu = [u0]

In [ ]:
for k in range(1000):
    uu.append(stormer_verlet_HH(0,uu[-1],h))

In [ ]:
uu = np.array(uu)
p2 = uu[:,1];
p2 = np.insert(p2,0,p20)
p2 = (p2[1:]+p2[:-1])/2.
q2 = uu[:,3]
plot_trajectory(p2,q2,skip=50)

In [ ]:
def HH_rhs(t,u):
    # u = [p1,p2,q1,q2]
    p1 = u[0]  # collocated at t_{n + 1/2}
    p2 = u[1]
    q1 = u[2]  # collocated at t_n
    q2 = u[3]
    dq1 = p1
    dq2 = p2
    dp1 = -(q1-2*q1*q2)
    dp2 = -(q2+q1**2+q2**2)
    return np.array([dp1,dp2,dq1,dq2])

In [ ]:
p10 = 0.
p20 = 0.
q10 = 0.0
q20 = 0.2
print(1./(0.5*(q10**2+q20**2+q10**2*q20 - 1./3.*q20**3)+0.5*(p10**2+p20**2)))
h = 0.05
p1half = p10 - h/2. * (q10-2*q10*q20)
p2half = p20 - h/2. * (q20+q10**2+q20**2)
u0 = np.array([p1half,p2half,q10,q20])
uu = [u0]

for k in range(1000):
    uu.append(explicit_euler(HH_rhs,0,uu[-1],h))

In [ ]:
uu = np.array(uu)
p2 = uu[:,1];
p2 = np.insert(p2,0,p20)
p2 = (p2[1:]+p2[:-1])/2.
q2 = uu[:,3]
plot_trajectory(p2,q2,skip=50)